In [1]:
#Shishir Khanal
#CMU-Optimal Controls from Jack Manchester
#Using convex optimization library to formulate a least squares optimization problem

In [4]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.add("Convex")
Pkg.add("ECOS")
using LinearAlgebra, Plots
import ForwardDiff as FD
using Test
import Convex as cvx 
import ECOS
using Random
Random.seed!(1)

  Activating project at `~/Documents/Optimal_Control/Sims/Untitled Folder`
   Resolving package versions...
  No Changes to `~/Documents/Optimal_Control/Sims/Untitled Folder/Project.toml`
  No Changes to `~/Documents/Optimal_Control/Sims/Untitled Folder/Manifest.toml`
   Resolving package versions...
   Installed ECOS_jll ─ v200.0.800+0
   Installed ECOS ───── v1.1.1
    Updating `~/Documents/Optimal_Control/Sims/Untitled Folder/Project.toml`
  [e2685f51] + ECOS v1.1.1
    Updating `~/Documents/Optimal_Control/Sims/Untitled Folder/Manifest.toml`
  [fa961155] + CEnum v0.4.2
  [e2685f51] + ECOS v1.1.1
  [c2c64177] + ECOS_jll v200.0.800+0
Precompiling project...
  ✓ ECOS_jll
  ✓ ECOS
  2 dependencies successfully precompiled in 66 seconds. 43 already precompiled.
[ Info: Precompiling ECOS [e2685f51-7e38-5353-a97d-a921fd2c8199]
┌ Warning: Module MathOptInterface with build ID 63145017145938 is missing from the cache.
│ This may mean MathOptInterface [b8f27783-ece8-5eb3-8dc8-9495eed66fee] d

TaskLocalRNG()

# Convex.jl tutorial

This is convex modeling tool in Julia that let's us write out problems in a simple way, and then Convex.jl transforms them and sends them off to be solved (we're using [ECOS](https://github.com/embotech/ecos) as our solver today). If you want examples/inspiration for this technology, there are a few like this:

- Python: [CVXPY](https://www.cvxpy.org/) or [CVXOPT](http://cvxopt.org/) (cvxpy is probably what you want)
- Matlab: [CVX](http://cvxr.com/cvx/) or [YALMIP](https://yalmip.github.io/) (I like CVX better)
- R: [CVXR](https://cvxr.rbind.io/)

For Convex.jl the [repo is here](https://github.com/jump-dev/Convex.jl), and the [docs are here](https://jump.dev/Convex.jl/stable/)

These tools are just used for formulating your problem and verifying that it is Convex. The problem itself is solved by one of many available solvers, many common ones are:

- OSQP
- ECOS (Can Handle Conic Constraints)
- CPLEX (Commercial)
- Mosek (Commercial)
- Gurobi (Commercial)
- COSMO 
- SeDuMi 
- SDPT3 
- GLPK 
- Hypatia 

## Least Squares 
For overdetermined systems (more equations than variables, "skinny" matrix A)
$$ \begin{align} \min_{x} \quad & \|Ax - b\|^2_2
 \end{align}$$

In [6]:
@testset "overdetermined" begin
    # overdetermined
    A = randn(10,5)
    b = randn(10)
    x = cvx.Variable(5)
    
    
    prob = cvx.minimize(cvx.sumsquares(A*x-b))  #sumsquares(y) = dot(y,y) = norm(y)^2
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix #This will always be a matrix
    xcvx = vec(x.value) #convert to vector easily
    
    #compare with pseudoinverse
    @test norm(xcvx - (A'*A\(A'*b))) < 1e-4
end

Test Summary:  | Pass  Total  Time
overdetermined |    1      1  0.0s

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.322e+00  +7e+00  5e-01  6e-02  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -8.212e-02  -2.569e-01  +1e+00  1e-01  1e-02  2e-01  5e-01  0.7834  2e-02   1  1  1 |  0  0
 2  +8.555e-01  +1.222e+00  +1e+00  1e+00  5e-02  3e+00  3e-01  0.5729  6e-01   2  2  2 |  0  0
 3  +2.382e+00  +2.671e+00  +2e-01  1e-01  6e-03  6e-01  5e-02  0.8540  1e-02   2  1  1 |  0  0
 4  +7.104e-01  +2.170e+00  +1e-01  3e-01  1e-02  2e+00  4e-02  0.4921  5e-01   2  2  2 |  0  0
 5  +3.248e+00  +3.096e+00  +1e-01  7e-02  3e-03  3e-02  3e-02  0.5470  6e-01   2  2  2 |  0  0
 6  +4.724e+00  +4.713e+00  +1e-02  1e-02  5e-04  2e-02  3e-03  0.9009  4e-03   2  2  2 |  0  0
 7  +5.226e+00  +5.243e+00  +2e-03  3e-03  1e-04  3e-02  6e-04  0.8823  

Test.DefaultTestSet("overdetermined", Any[], 1, false, false, true, 1.681956546396043e9, 1.681956546399181e9)

In [8]:
@testset "underdetermined" begin
    #overdetermined
    A = randn(5,10)
    b = randn(5)
    x = cvx.Variable(10)
    prob = cvx.minimize(cvx.sumsquares(x))
        
    #add constraint
    prob.constraints += (A*x == b)
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix #This will always be a matrix
    xcvx = vec(x.value) #convert to vector easily
    
    #compare with pseudoinverse
    @test norm(xcvx - A'*((A*A')\b)) < 1e-4
end

Test Summary:   | Pass  Total  Time
underdetermined |    1      1  0.4s

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.322e+00  +6e+00  4e-01  7e-02  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -9.555e-02  -1.486e-01  +5e-01  3e-02  4e-03  1e-01  2e-01  0.9187  2e-02   1  1  1 |  0  0
 2  -1.315e-01  -1.136e-01  +5e-01  1e-01  7e-03  4e-01  2e-01  0.3838  5e-01   2  2  2 |  0  0
 3  +2.373e-01  +1.775e-01  +1e-01  2e-02  1e-03  3e-03  4e-02  0.9221  2e-01   2  1  2 |  0  0
 4  +2.708e-01  +2.648e-01  +1e-02  2e-03  1e-04  6e-04  3e-03  0.9142  1e-03   2  2  2 |  0  0
 5  +2.860e-01  +2.857e-01  +1e-03  3e-04  1e-05  5e-04  3e-04  0.9453  5e-02   2  2  2 |  0  0
 6  +2.873e-01  +2.872e-01  +2e-04  4e-05  2e-06  7e-05  4e-05  0.8726  9e-03   2  1  2 |  0  0
 7  +2.876e-01  +2.876e-01  +6e-06  2e-06  9e-08  5e-06  2e-06  0.9876

Test.DefaultTestSet("underdetermined", Any[], 1, false, false, true, 1.681956941533413e9, 1.681956941960012e9)

## Equality constrained QP 

$$ \begin{align} \min_{x} \quad & \frac{1}{2} x^TQx + q^Tx \\ 
 \text{st} \quad & A x = b 
 \end{align}$$

In [10]:
let
    
    n = 10
    Q = randn(n,n); Q = Q'*Q + I # create PSD matrix
    q = randn(n)
    
    A = randn(3, n)
    b = randn(3)
    
    x = cvx.Variable(n)
    
    #NOTE: quadform(x, Q) = x'*Q*x
    cost = 0.5*cvx.quadform(x, Q) + dot(q,x)
    
    prob = cvx.minimize(cost)
    
    prob.constraints += (A*x == b)
    
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix #This will always be a matrix
    xcvx = vec(x.value) #convert to vector easily
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.854e-01  -2.609e+01  +9e+01  4e-01  3e-01  1e+00  2e+01    ---    ---    1  2  - |  -  - 
 1  -6.341e-01  -1.922e+00  +8e+00  2e-02  2e-02  5e-01  2e+00  0.9243  3e-02   2  2  2 |  0  0
 2  -8.744e-01  -1.611e+00  +5e+00  2e-02  1e-02  9e-01  1e+00  0.6830  4e-01   2  2  2 |  0  0
 3  -4.956e-01  -5.643e-01  +5e-01  2e-03  1e-03  8e-02  1e-01  0.9890  1e-01   2  2  2 |  0  0
 4  -4.539e-01  -4.568e-01  +1e-02  4e-05  3e-05  9e-04  4e-03  0.9730  1e-04   2  2  2 |  0  0
 5  -4.533e-01  -4.537e-01  +2e-03  6e-06  5e-06  1e-04  5e-04  0.8632  2e-03   2  2  2 |  0  0
 6  -4.533e-01  -4.534e-01  +4e-04  1e-06  1e-06  5e-05  1e-04  0.8797  1e-01   2  1  1 |  0  0
 7  -4.533e-01  -4.533e-01  +4e-05  1e-07  1e-07  5e-06  1e-05  0.9031  8e-03   3  1  1 |  0  0
 8  -4.533e-01  -4.533e-01  +8e-06  3e-08  2e-

10-element Vector{Float64}:
 -0.4487687765255043
  0.6107400162997437
  0.03556628269949974
 -0.024201951782730602
  0.050273990447075895
  0.21195899494287954
  0.04527270874277189
  0.1550311834360013
  0.348614722299006
 -0.08023466974569989

## Letting Convex.jl do the parsing 

$$ \begin{align} \min_{x} \quad & \|Ax - b\|_1 \\ 
 \text{st} \quad &\|x\|_2 \leq 3
 \end{align}$$
 
 This problem is not in any sort of "standard form", but it is convex. We will let Convex.jl will convert this into a standard form "canonicalizing it", and send it ECOS to solve. 

In [11]:
let 
    A = randn(10,5)
    b = randn(10)
    x = cvx.Variable(5)
    
    prob = cvx.minimize(norm(A*x - b, 1))
    prob.constraints += (norm(x,2) <= 3)
    
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    xcvx = x.value::Matrix #This will always be a vector
    scvx = vec(x.value) #convert to vector easily
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.070e-16  -3.000e+00  +6e+01  5e-01  6e-01  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  +2.493e+00  +2.597e+00  +2e+01  1e-01  1e-01  9e-01  9e-01  0.7644  1e-01   1  1  1 |  0  0
 2  +2.903e+00  +2.939e+00  +3e+00  1e-02  1e-02  1e-01  1e-01  0.8638  2e-02   1  1  1 |  0  0
 3  +2.961e+00  +2.965e+00  +5e-01  2e-03  3e-03  2e-02  3e-02  0.8274  3e-02   1  1  1 |  0  0
 4  +2.988e+00  +2.988e+00  +5e-02  2e-04  2e-04  2e-03  2e-03  0.9298  2e-02   1  1  1 |  0  0
 5  +2.989e+00  +2.989e+00  +5e-04  2e-06  2e-06  2e-05  3e-05  0.9890  1e-04   1  1  1 |  0  0
 6  +2.989e+00  +2.989e+00  +6e-06  2e-08  3e-08  2e-07  3e-07  0.9890  1e-04   1  1  1 |  0  0
 7  +2.989e+00  +2.989e+00  +6e-08  2e-10  3e-10  2e-09  3e-09  0.9890  1e-04   1  0  0 |  0  0
 8  +2.989e+00  +2.989e+00  +7e-10  3e-12  3e-

5-element Vector{Float64}:
 -0.18786716134917025
 -0.3681298788356769
  0.05329757748782587
 -0.38626872563229914
 -0.03692377234157958

## Convex Trajectory Optimization
$$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & \sum_{i=1}^{N-1} \bigg[ \|x_i - x_g\|_2^2 + \|u_i\|_1 \bigg] + \frac{1}{2}x_N^TQ_fx_N & \\ 
 \text{st} \quad & x_1 = x_{\text{IC}} \\ 
 & x_{i+1} = A x_i + Bu_i \quad &\text{for } i = 1,2,\ldots,N-1 \\ 
 & x_N = x_g \\ 
 & \|u_i\|_2 \leq 3 \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 & x_{min} \leq x_i \leq x_{max} \quad &\text{for } i = 1,2,\ldots,N-1\\ 
 \end{align}$$

In [16]:
function controllable(A,B)
    n = size(A, 1)
    C = hcat([A^i*B for i = 0:(n-1)]...)
    return rank(C) == n
end

let
    #Create a linear system
    nx = 4
    nu = 2
    A = randn(nx,nx);
    B = randn(nx, nu);
    @assert controllable(A,B)
    
    #time step
    N = 20
    x_ic = randn(nx)
    x_g = randn(nx)
    
    #terminal cost
    Qf = randn(nx, nx); Qf = Qf'*Qf + I #make PSD Qf
    
    #create cvx variables x_k = X[:,k], u_k = U[:,k]
    X = cvx.Variable(nx, N)
    U = cvx.Variable(nu, N - 1)
    
    #create cost
    cost = 0
    for k = 1:(N-1)
        xk = X[:, k]
        uk = U[:, k]
        cost += cvx.sumsquares(xk - x_g)
        cost += norm(uk, 1)
    end
    xn = X[:, N]
    cost += 0.5*cvx.quadform(xn, Qf)
    
    #initialize CVX problem
    prob = cvx.minimize(cost)
    
    #initial condition constraint
    prob.constraints += X[:, 1] == x_ic
    
    for k = 1:(N-1)
        #dynamics constraints
        prob.constraints += (X[:,k+1] == A*X[:,k] + B*U[:,k])
    end
    
    #goal constraint
    prob.constraints += X[:, N] == x_g
    
    #norm(u)<3
    for k = 1:(N-1)
        uk = U[:,k]
        prob.constraints += norm(uk, 2) <= 3
    end
    
    x_min = -20*ones(nx)
    x_max = 20*ones(nx)
    for k = 1:N
        xk = X[:,k]
        prob.constraints += xk <= x_max
        prob.constraints += xk >= x_min
    end
    
    #solve problem (silent solver tells us the output)
    cvx.solve!(prob, ECOS.Optimizer; silent_solver = false)
    
    if prob.status != cvx.MathOptInterface.OPTIMAL
        error("Convex.jl problem failed to solve for some reason")
    end
    
    #convert the solution matrices into vectors of vectors
    X = X.value::Matrix
    U = U.value::Matrix
end


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -4.074e+03  +6e+03  1e-01  5e-01  1e+00  2e+01    ---    ---    1  2  - |  -  - 
 1  +3.919e+01  -9.878e+02  +2e+03  2e-02  8e-02  4e-01  5e+00  0.7233  2e-02   1  1  1 |  0  0
 2  +5.259e+01  -9.693e+02  +2e+03  2e-02  7e-02  7e-01  5e+00  0.1129  7e-01   2  1  2 |  0  0
 3  +7.074e+01  -9.124e+02  +1e+03  2e-02  6e-02  1e+00  4e+00  0.1970  7e-01   1  1  2 |  0  0
 4  +1.293e+02  -7.056e+02  +1e+03  2e-02  4e-02  2e+00  3e+00  0.6473  7e-01   1  1  1 |  0  0
 5  +1.115e+02  -4.254e+02  +8e+02  1e-02  2e-02  2e+00  2e+00  0.4622  2e-01   2  1  2 |  0  0
 6  +1.200e+02  -3.765e+02  +7e+02  1e-02  2e-02  2e+00  2e+00  0.2777  7e-01   2  1  2 |  0  0
 7  +1.251e+02  -3.104e+02  +6e+02  1e-02  1e-02  2e+00  2e+00  0.3235  5e-01   2  1  2 |  0  0
 8  +1.277e+02  -1.019e+02  +3e+02  5e-03  7e-

2×19 Matrix{Float64}:
 -0.136922  -1.83035  -3.0         …  -2.05116   -2.21177     -1.32521
  1.3179    -2.37693  -2.1528e-11      0.296269   1.23485e-8   1.32907e-9